### Imports

In [8]:
from bs4 import BeautifulSoup
import requests
import json
import re
from pymongo import MongoClient
from nltk.corpus import stopwords
import pyarabic.araby as araby
import pyarabic.number as number
import string

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

In [10]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    link = article.find(name='a').get('href')
    title = article.find(name='a').get('title')
    articals.append({title : link})

In [11]:
with open('hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})

    with open('hess_article_links.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [12]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

In [13]:
with open('hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})

    with open('hess_article_content.json', 'w', encoding='utf-8') as f:
        json.dump(contents, f, ensure_ascii=False, indent=2)

In [14]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

In [135]:
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
    text = art_content[0]['content']
    text.encode('utf-8')
    print(text)

أفاد مجلس المنافسة بأن المبيعات السنوية من الكازوال والبنزين وصلت إلى ما يقارب 6.78 مليون طن، أي ما يقارب 8.12 مليار لتر في سنة 2023، مقابل 6.53 مليون طن (حوالي 7.83 مليار لتر) في سنة 2022، وهو ما يمثل زيادة بحوالي 3.75 في المائة.وأوضح مجلس المنافسة، في تقرير حول تتبع الالتزامات المتعهد بها من قبل شركات التوزيع بالجملة للكازوال والبنزين، في إطار اتفاقيات المعاملات المبرمة مع المجلس، أنه بالنسبة للمبيعات من حيث القيمة، وفي غياب البيانات المتعلقة برقم المعاملات الإجمالي المحقق في سوق التوزيع لجميع الفاعلين مجتمعين، فوحده رقم معاملات الشركات التسع المعنية، التي تمثل ما يقرب من 90 في المئة من إجمالي الواردات و83 في المئة من الكميات المباعة، مأخوذ في الاعتبار.وبلغ إجمالي رقم معاملات الشركات التسع، المتحقق في قطاع توزيع الكازوال والبنزين (باستثناء الوقود البحري) 79 مليار درهم في سنة 2023 مقابل 85 مليار درهم في سنة 2022، أي بانخفاض بنحو 7 في المئة.وفي التفاصيل، يتيح تحليل توزيع المبيعات من حيث الحجم حسب شريحة الزبناء لسنة 2023 الإشارة إلى أن شبكة محطات الخدمة تمثل غالبية مبيعات الكازوال والبن

### Normalization

In [136]:
for form in ('أ', 'إ', 'آ',):
    norm_text = text.replace(form, 'ا')

norm_text = norm_text.replace('ة', 'ه')

for form in ('ؤ', 'ئ',):
    norm_text = norm_text.replace(form, 'ء')
    
print("Normalized text: ", norm_text)

Normalized text:  أفاد مجلس المنافسه بأن المبيعات السنويه من الكازوال والبنزين وصلت إلى ما يقارب 6.78 مليون طن، أي ما يقارب 8.12 مليار لتر في سنه 2023، مقابل 6.53 مليون طن (حوالي 7.83 مليار لتر) في سنه 2022، وهو ما يمثل زياده بحوالي 3.75 في الماءه.وأوضح مجلس المنافسه، في تقرير حول تتبع الالتزامات المتعهد بها من قبل شركات التوزيع بالجمله للكازوال والبنزين، في إطار اتفاقيات المعاملات المبرمه مع المجلس، أنه بالنسبه للمبيعات من حيث القيمه، وفي غياب البيانات المتعلقه برقم المعاملات الإجمالي المحقق في سوق التوزيع لجميع الفاعلين مجتمعين، فوحده رقم معاملات الشركات التسع المعنيه، التي تمثل ما يقرب من 90 في المءه من إجمالي الواردات و83 في المءه من الكميات المباعه، مأخوذ في الاعتبار.وبلغ إجمالي رقم معاملات الشركات التسع، المتحقق في قطاع توزيع الكازوال والبنزين (باستثناء الوقود البحري) 79 مليار درهم في سنه 2023 مقابل 85 مليار درهم في سنه 2022، أي بانخفاض بنحو 7 في المءه.وفي التفاصيل، يتيح تحليل توزيع المبيعات من حيث الحجم حسب شريحه الزبناء لسنه 2023 الإشاره إلى أن شبكه محطات الخدمه تمثل غالبيه مبي

### Removing punctuation and tokenization

In [137]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['أفاد', 'مجلس', 'المنافسة', 'بأن', 'المبيعات', 'السنوية', 'من', 'الكازوال', 'والبنزين', 'وصلت', 'إلى', 'ما', 'يقارب', '6', '.', '78', 'مليون', 'طن', '،', 'أي', 'ما', 'يقارب', '8', '.', '12', 'مليار', 'لتر', 'في', 'سنة', '2023', '،', 'مقابل', '6', '.', '53', 'مليون', 'طن', '(', 'حوالي', '7', '.', '83', 'مليار', 'لتر', ')', 'في', 'سنة', '2022', '،', 'وهو', 'ما', 'يمثل', 'زيادة', 'بحوالي', '3', '.', '75', 'في', 'المائة', '.', 'وأوضح', 'مجلس', 'المنافسة', '،', 'في', 'تقرير', 'حول', 'تتبع', 'الالتزامات', 'المتعهد', 'بها', 'من', 'قبل', 'شركات', 'التوزيع', 'بالجملة', 'للكازوال', 'والبنزين', '،', 'في', 'إطار', 'اتفاقيات', 'المعاملات', 'المبرمة', 'مع', 'المجلس', '،', 'أنه', 'بالنسبة', 'للمبيعات', 'من', 'حيث', 'القيمة', '،', 'وفي', 'غياب', 'البيانات', 'المتعلقة', 'برقم', 'المعاملات', 'الإجمالي', 'المحقق', 'في', 'سوق', 'التوزيع', 'لجميع', 'الفاعلين', 'مجتمعين', '،', 'فوحده', 'رقم', 'معاملات', 'الشركات', 'التسع', 'المعنية', '،', 'التي', 'تمثل', 'ما', 'يقرب', 'من', '90', 'في'

In [138]:
ar_punct = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [139]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['أفاد', 'مجلس', 'المنافسة', 'بأن', 'المبيعات', 'السنوية', 'من', 'الكازوال', 'والبنزين', 'وصلت', 'إلى', 'ما', 'يقارب', '6', '78', 'مليون', 'طن', 'أي', 'ما', 'يقارب', '8', '12', 'مليار', 'لتر', 'في', 'سنة', '2023', 'مقابل', '6', '53', 'مليون', 'طن', 'حوالي', '7', '83', 'مليار', 'لتر', 'في', 'سنة', '2022', 'وهو', 'ما', 'يمثل', 'زيادة', 'بحوالي', '3', '75', 'في', 'المائة', 'وأوضح', 'مجلس', 'المنافسة', 'في', 'تقرير', 'حول', 'تتبع', 'الالتزامات', 'المتعهد', 'بها', 'من', 'قبل', 'شركات', 'التوزيع', 'بالجملة', 'للكازوال', 'والبنزين', 'في', 'إطار', 'اتفاقيات', 'المعاملات', 'المبرمة', 'مع', 'المجلس', 'أنه', 'بالنسبة', 'للمبيعات', 'من', 'حيث', 'القيمة', 'وفي', 'غياب', 'البيانات', 'المتعلقة', 'برقم', 'المعاملات', 'الإجمالي', 'المحقق', 'في', 'سوق', 'التوزيع', 'لجميع', 'الفاعلين', 'مجتمعين', 'فوحده', 'رقم', 'معاملات', 'الشركات', 'التسع', 'المعنية', 'التي', 'تمثل', 'ما', 'يقرب', 'من', '90', 'في', 'المئة', 'من', 'إجمالي', 'الواردات', 'و83', 'في', 'المئة', 'من', 'ال

In [140]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]
print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'أوت', 'حمو', 'ج', 'كأنّ', 'شرع', 'منها', 'مساء', 'بنا', 'نحن', 'نا', 'غ', 'لئن', 'تانِ', 'ثمانون', 'ثمانمئة', 'حتى', 'ممن', 'خ', 'همزة', 'رأى', 'إنا', 'أمسى', 'فيما', 'ثمّة', 'كلما', 'مافتئ', 'له', 'لسنا', 'عشرين', 'اللتان', 'راء', 'مرّة', 'نَخْ', 'ه', 'سابع', 'إياهما', 'تِه', 'كأين', 'هنالك', 'خلافا', 'سرا', 'كأي', 'إليكَ', 'آض', 'لات', 'أبدا', 'سوى', 'غادر', 'ثلاثاء', 'ستة', 'فمن', 'منه', 'ذواتي', 'سبعمائة', 'هذي', 'عشرة', 'أقل', 'حدَث', 'هما', 'خمسة', 'راح', 'ف', 'ظاء', 'أنشأ', 'اللتين', 'زاي', 'الآن', 'هاكَ', 'حيثما', 'إن', 'ذه', 'اثنان', 'طالما', 'هَاتَيْنِ', 'هَذِه', 'أجل', 'تلكما', 'سنتيم', 'ة', 'لما', 'حار', 'وُشْكَانَ', 'عدَّ', 'نوفمبر', 'سبعمئة', 'اربعون', 'لا سيما', 'هاتان', 'أكثر', 'لكن', 'إمّا', 'عشرون', 'لام', 'تينك', 'كى', 'ته', 'مكانكم', 'أن', 'إياهن', 'نيف', 'عين', 'ذان', 'ثامن', 'اللذين', 'تلكم', 'هؤلاء', 'لمّا', 'اثنا', 'إيانا', 'طَق', 'نيسان', 'تلك', 'ثلاث', 'رجع', 'منذ', 'إذما', 'شباط', 'ليسا', 'باء', 'يورو', 'دونك', 'عجبا', 'به', 'تسعمئة', 'قام', 'غ